<a href="https://colab.research.google.com/github/ambitama-poddar/Machine-Learning-Practicals/blob/main/DA_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Flatten ,Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip drive/My\ Drive/Colab\ Notebooks/fruits-360.zip

Streaming output truncated to the last 5000 lines.
  inflating: fruits-360/Training/Tomato 2/r2_280_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_281_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_282_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_283_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_284_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_285_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_286_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_287_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_289_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_29_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_290_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_293_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_294_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_295_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_296_100.jpg  
  inflating: fruits-360/Training/Tomato 2/r2_298_100.jpg  
  infl

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # All images will be rescaled by 1./255
# train_datagen = ImageDataGenerator(
#         rescale=1./255)

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale = 1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/fruits-360/Training',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        class_mode='categorical')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/content/fruits-360/Test',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        class_mode='categorical')

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [5]:
model = tf.keras.applications.InceptionV3(weights='imagenet',include_top=False, input_shape = (300, 300, 3))
#Adding custom Layers
x = model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
predictions = tf.keras.layers.Dense(131, activation="softmax")(x)
model = tf.keras.Model(inputs = model.input, outputs = predictions)
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                  

In [6]:
model_history = model.fit(
train_generator,
epochs = 10,
validation_data = validation_generator)

/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/10
  21/2116 [..............................] - ETA: 22:44:32 - loss: 15.0356 - accuracy: 0.0149

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Val accuracy')
plt.title('Training Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Val Loss')
plt.title('Training Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.show()